In [1]:
# 재시작 없이 import 파일 자동 적용
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
import pandas as pd
from scipy import interpolate
import neurokit2 as nk
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import h5py

# Data Loading

In [6]:
with h5py.File("./dataset/KMedicon/features.h5", 'r') as f:
    ecg = f['ecg'][:]
    label = f['label'][:]

In [7]:
# resampling to 250Hz
def resampling(array, freq, kind='linear'):
    t = np.linspace(1, len(array), len(array))
    f = interpolate.interp1d(t, array, kind=kind)
    t_new = np.linspace(1, len(array), int(len(array)/freq * 250))
    new_array = f(t_new)
    return new_array


# standard normalization 
def normalize(data):
    scaler = StandardScaler()
    data_norm = scaler.fit_transform(data)
    return data_norm


# function of R peaks of a resampled trial
def R_Peaks(ecg_data):
    # get R Peak positions
    pos = []
    # get R Peak intervals
    trial_interval = []
    for ch in range(ecg_data.shape[1]):
        cleaned_ecg = nk.ecg_clean(ecg_data[:, ch], sampling_rate=250, method='neurokit')
        signals, info = nk.ecg_peaks(cleaned_ecg, sampling_rate=250, correct_artifacts=False)
        peaks = signals[signals['ECG_R_Peaks']==1].index.to_list()
        pos.append(peaks)
        channel_interval = []
        for i in range(len(peaks)-1):
            channel_interval.append(peaks[i+1] - peaks[i])
        trial_interval.append(channel_interval)
        
    df_peaks = pd.DataFrame(pos) # [C=15, num of the R-Peaks of a channel]
    df = pd.DataFrame(trial_interval).T
    med = df.median()
    return df, med, df_peaks

In [10]:
ecg.shape

(2400, 5000, 12)

In [11]:
ecg[0].shape

(5000, 12)

In [24]:
cnt = 0
p_ecg = []

for ecg_data in tqdm(ecg):
    sub = []
    trial = []
    for ch in range(ecg_data.shape[1]):
        data = resampling(ecg_data[:,ch], freq=500, kind='linear')
        # data = ecg_data[:, ch]
        trial.append(data)
        
    trial = np.array(trial).T
    # trial_norm = normalize(trial)
    sub.append(trial)

    sub = np.array(sub)
    sub = sub.reshape(-1, 250, sub.shape[-1])  # split 10s trial into 1s sample

    p_ecg.append(sub)
    # np.save('./dataset/KMedicon/Feature/feature_{}.npy'.format(cnt), sub)

    cnt +=1

print(sub.shape)

100%|██████████| 2400/2400 [00:05<00:00, 402.31it/s]

(10, 250, 12)


In [28]:
np.array(p_ecg).shape

(2400, 10, 250, 12)

In [29]:
with h5py.File("./processed_features.h5", 'w') as f:
    f.create_dataset('ecg', data=np.array(p_ecg))
    f.create_dataset('label', data=np.array(label))

# pkl to h5

In [ ]:
with open("./dataset/Signal_Test_Public.pkl", 'rb') as f:
    signal = pickle.load(f)

In [5]:
with h5py.File("./features.h5", 'w') as f:
    f.create_dataset('ecg', data=np_signal)
    # f.create_dataset('label', data=np.array(label))